In [99]:
import s3fs
import hvac
import os

client = hvac.Client(url='https://vault.lab.sspcloud.fr',
                     token=os.environ['VAULT_TOKEN'])
secret = os.environ['VAULT_MOUNT'] + os.environ['VAULT_TOP_DIR'] + '/s3_creds'
mount_point, secret_path = secret.split('/', 1)
secret_dict = client.secrets.kv.read_secret_version(path=secret_path, mount_point = mount_point)
os.environ["AWS_ACCESS_KEY_ID"] = secret_dict['data']['data']['ACCESS_KEY']
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_dict['data']['data']['SECRET_KEY']
try:
    del os.environ['AWS_SESSION_TOKEN']
except KeyError:
    pass

In [100]:
import pickle
import json
import mlflow
import os
import sys
import re
import unidecode
from nltk.corpus import stopwords as ntlk_stopwords
from nltk.stem.snowball import SnowballStemmer
sys.path.append("../src/page_selection/")
from tqdm import tqdm
from time import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from utils import (
    clean_page_content,
    extract_document_content,
    fit_transform_vectorizer,
    train_random_forest,
    get_numeric_char_rate
)
from model_wrapper import RandomForestWrapper
import pandas as pd

In [101]:
def load_extra_labeled_data_checked():
    """ """

    with fs.open(
        "s3://projet-extraction-tableaux/data/df_train_rf_corr.pickle", "rb"
    ) as f:
        df = pickle.load(f)

    flat_corpus = list(df.text)
    flat_corpus_with_number = [
        clean_page_content(page) for page in flat_corpus
    ]
    flat_corpus = [remove_number(page) for page in flat_corpus_with_number]
    valid_labels = list(df.tableau_f_et_p)

    num_rates = []
    num_rates = [
        get_numeric_char_rate(content) for content in flat_corpus_with_number
    ]

    return flat_corpus, valid_labels, num_rates

In [111]:
fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://" + os.environ["AWS_S3_ENDPOINT"]},
    key=os.environ["AWS_ACCESS_KEY_ID"],
    secret=os.environ["AWS_SECRET_ACCESS_KEY"],
)

In [118]:
with fs.open("s3://projet-extraction-tableaux/data/df_train_rf_corr.pickle", "rb") as f:
    df = pickle.load(f)

PermissionError: Forbidden

In [113]:
df

,siren,pagenumber,text,tableau_f_et_p,accOCR,tableauPDF
0,5820378,1,RCS : AMIENS Code greffe : 8002 Documents comp...,0,85.776467,1
1,5820378,2,GREFFE DU TRIBUNAL DE COMMERCE …… D'AMIENS Dat...,0,91.648514,1
2,5820378,3,...,0,NaN,1
3,5820378,4,...,0,NaN,1
4,5820378,5,...,0,NaN,1
...,...,...,...,...,...,...
19852,998823504,45,4.9. Analyse et échéance des dettes\r\r\n...,0,NaN,1
19853,998823504,46,- de remboursement de pénalités et intérêts...,0,NaN,1
19854,998823504,47,6.2.2. ...,0,NaN,1
19855,998823504,48,ADECCO FRANCE Société par actions simplifiée a...,0,91.764892,1


In [114]:
df.accOCR = pd.to_numeric(df.accOCR.replace(',', '.', regex=True))

In [115]:
seuil = 70

In [116]:
test = df[(df.accOCR.isnull()) | (df.accOCR > seuil)]

In [117]:
test

,siren,pagenumber,text,tableau_f_et_p,accOCR,tableauPDF
0,5820378,1,RCS : AMIENS Code greffe : 8002 Documents comp...,0,85.776467,1
1,5820378,2,GREFFE DU TRIBUNAL DE COMMERCE …… D'AMIENS Dat...,0,91.648514,1
2,5820378,3,...,0,NaN,1
3,5820378,4,...,0,NaN,1
4,5820378,5,...,0,NaN,1
...,...,...,...,...,...,...
19852,998823504,45,4.9. Analyse et échéance des dettes\r\r\n...,0,NaN,1
19853,998823504,46,- de remboursement de pénalités et intérêts...,0,NaN,1
19854,998823504,47,6.2.2. ...,0,NaN,1
19855,998823504,48,ADECCO FRANCE Société par actions simplifiée a...,0,91.764892,1


In [132]:
def test(b,c,a=2):
    return -a+b+c

In [135]:
test(1,6)

5